In [1]:
!pip install gradio pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

Saving final_model.pkl to final_model.pkl


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle

# Load the model
with open('final_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Define the prediction function
def predict_seriousness(age, sex, symptom_text, died, l_threat, hospital, hospdays, disable, recovd,
                        other_meds, cur_ill, history, birth_defect, allergies, vaccine, manufacturer,
                        symptom1, symptom2, symptom3, symptom4, symptom5):

    # Convert string inputs to binary
    sex = 1 if sex == 'Male' else 0
    died = 1 if died == 'Yes' else 0
    l_threat = 1 if l_threat == 'Yes' else 0
    hospital = 1 if hospital == 'Yes' else 0
    disable = 1 if disable == 'Yes' else 0
    recovd = 1 if recovd == 'Yes' else 0
    birth_defect = 1 if birth_defect == 'Yes' else 0

    # Create DataFrame
    new_data = pd.DataFrame({
        'AGE_YRS': [age],
        'SEX': [sex],
        'SYMPTOM_TEXT': [symptom_text],
        'DIED': [died],
        'L_THREAT': [l_threat],
        'HOSPITAL': [hospital],
        'HOSPDAYS': [hospdays],
        'DISABLE': [disable],
        'RECOVD': [recovd],
        'OTHER_MEDS': [other_meds],
        'CUR_ILL': [cur_ill],
        'HISTORY': [history],
        'BIRTH_DEFECT': [birth_defect],
        'ALLERGIES': [allergies],
        'VAX_MANU': [manufacturer],
        'SYMPTOM1': [symptom1],
        'SYMPTOM2': [symptom2],
        'SYMPTOM3': [symptom3],
        'SYMPTOM4': [symptom4],
        'SYMPTOM5': [symptom5],
        'VAX_TYPE_COVID': [vaccine == 'COVID'],
        'VAX_TYPE_FLU': [vaccine == 'FLU'],
        'VAX_TYPE_PPV': [vaccine == 'PPV'],
        'VAX_TYPE_VARZOS': [vaccine == 'VARZOS'],
        'CSL': [manufacturer == 'CSL'],
        'GLAXOSMITHKLINE': [manufacturer == 'GLAXOSMITHKLINE'],
        'JANSSEN': [manufacturer == 'JANSSEN'],
        'MEDIMMUNE': [manufacturer == 'MEDIMMUNE'],
        'MERCK': [manufacturer == 'MERCK'],
        'MODERNA': [manufacturer == 'MODERNA'],
        'NOVARTIS': [manufacturer == 'NOVARTIS'],
        'NOVAVAX': [manufacturer == 'NOVAVAX'],
        'PFIZER\\BIONTECH': [manufacturer == 'PFIZER\\BIONTECH'],
        'WYETH': [manufacturer == 'WYETH'],
        'PROTEIN_SCIENCES': [manufacturer == 'PROTEIN_SCIENCES'],
        'SANOFI': [manufacturer == 'SANOFI'],
        'SEQIRUS': [manufacturer == 'SEQIRUS'],
        'UNKNOWN MANUFACTURER': [manufacturer == 'UNKNOWN MANUFACTURER'],
    })

    # Replace missing values for object columns
    for col in ['OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'ALLERGIES']:
        new_data[col] = new_data[col].replace(np.nan, '', regex=True)

    # Ensure new_data only contains columns used in training
    expected_columns = loaded_model.feature_names_in_
    for col in expected_columns:
        if col not in new_data.columns:
            new_data[col] = 0
    new_data = new_data[expected_columns]

    # Predict
    prediction = loaded_model.predict(new_data)
    return "Serious" if prediction[0] == 1 else "Not Serious"


In [8]:
# Define the Gradio Interface
interface = gr.Interface(
    fn=predict_seriousness,
    inputs=[
        gr.Number(label="Age"),
        gr.Radio(choices=["Male", "Female"], label="Sex"),
        gr.Textbox(label="Symptom Text"),
        gr.Radio(choices=["Yes", "No"], label="Died"),
        gr.Radio(choices=["Yes", "No"], label="Was the vaccination Life Threatening?"),
        gr.Radio(choices=["Yes", "No"], label="Were You Hospitalized?"),
        gr.Number(label="Hospital Days"),
        gr.Radio(choices=["Yes", "No"], label="Did Vaccination cause Disabilty?"),
        gr.Radio(choices=["Yes", "No"], label="Were you Recovered?"),
        gr.Textbox(label="Do you take any other medications?"),
        gr.Textbox(label="Do you have Current Illness?"),
        gr.Textbox(label="Any Medical History?"),
        gr.Radio(choices=["Yes", "No"], label="Did Vaccination cause Birth Defect?"),
        gr.Textbox(label="Do you have any allergies?"),
        gr.Dropdown(choices=['COVID', 'FLU', 'PPV', 'VARZOS'], label="Vaccine Type"),
        gr.Dropdown(
            choices=['MERCK', 'PFIZER\\BIONTECH', 'MODERNA', 'JANSSEN', 'UNKNOWN MANUFACTURER', 'CSL', 'GLAXOSMITHKLINE',
                     'MEDIMMUNE', 'NOVARTIS', 'NOVAVAX', 'WYETH', 'PROTEIN_SCIENCES', 'SANOFI', 'SEQIRUS'],
            label="Vaccine Manufacturer"
        ),
        gr.Textbox(label="Primary Symptom"),
        gr.Textbox(label="Secondary Symptom"),
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="Vaccine Adverse Event Seriousness Predictor",
    description="Predict the seriousness of vaccine adverse events based on patient and vaccine details.",
)

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b9da073d07c5da326.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
